In [40]:
import os
%pwd

'/Users/vishalchandru/MacBook/KidneyCNN'

In [28]:
os.chdir('KidneyCNN')

In [41]:
%pwd

'/Users/vishalchandru/MacBook/KidneyCNN'

In [42]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [43]:
from Kidney_CTScan.utils.utility import read_yaml, create_directories
from Kidney_CTScan.constants import *
import tensorflow as tf
import os

In [44]:
class ConfigurationManager():
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAM_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training_config = self.config.training
        prepare_base_model_config = self.config.prepare_base_model
        data_ingestion_config = self.config.data_ingestion
        params = self.params

        training_data = os.path.join(data_ingestion_config.unzip_dir, "KidneyCTScan")

        create_directories([Path(training_config.root_dir)])

        training_config = TrainingConfig(
                                    root_dir = Path(training_config.root_dir),
                                    trained_model_path = Path(training_config.trained_model_path),
                                    updated_base_model_path = Path(prepare_base_model_config.updated_base_model_path),
                                    training_data = Path(training_data),
                                    params_epochs = params.EPOCHS,
                                    params_batch_size = params.BATCH_SIZE,
                                    params_is_augmentation = params.AUGMENTATION,
                                    params_image_size = params.IMAGE_SIZE
                                )
        
        return training_config

In [45]:
import os
from zipfile import ZipFile
import tensorflow as tf
import time

In [46]:
class Training:
    def __init__(self, config = TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
    
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        #generate validation data
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

        #generate training data with or without augmentation
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs
        )
    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator
        )

        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    

In [47]:
from Kidney_CTScan import logger, CustomException
import sys

In [48]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise CustomException(e,sys)

[ 2023-11-28 17:45:03,218 ] INFO : utility - 20 - Read the yaml file from config/config.yaml
[ 2023-11-28 17:45:03,220 ] INFO : utility - 20 - Read the yaml file from params.yaml
[ 2023-11-28 17:45:03,221 ] INFO : utility - 31 - Created a directory on artifacts
[ 2023-11-28 17:45:03,221 ] INFO : utility - 31 - Created a directory on artifacts/model_training
Found 211 images belonging to 2 classes.
Found 851 images belonging to 2 classes.
53/53 [==============================] - 2311s 44s/step - loss: 15.0354 - accuracy: 0.5126 - val_loss: 0.3852 - val_accuracy: 0.9231


/Users/vishalchandru/MacBook/KidneyCNN/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
